In [1]:
def Invert_Dict(dictionary):
    exit_dict = {}
    for i in dictionary:
        for j in dictionary[i]:
            if j in exit_dict:
                exit_dict[j].append(i)
            else:
                exit_dict[j] = [i]
    return exit_dict

from tabulate import tabulate  #украшательства
import copy

def Print_2D_Table(list_2d, symbols_to_delete = "[]' "):
    to_print = copy.deepcopy(list_2d)
    for i in range(len(to_print)):          
        for j in range(len(to_print[i])):
            if to_print[i][j] == []:
                to_print[i][j] = ""
            else:
                cell = str(to_print[i][j])
                for k in symbols_to_delete:
                    cell = cell.replace(k, "")
                to_print[i][j] = cell

    print(tabulate(to_print, stralign="center", tablefmt="fancy_grid")) #шедевр
    
def Add_Row_Column_Numeration(list_2d, start_col = 0, start_row = 0):
    table = copy.deepcopy(list_2d)
    for i in range(len(table)):
        table[i].insert(0, i + start_col)
    table.insert(0, [i+start_row-1 for i in range(len(table[0]))])
    table[0][0] = ""
    
    return table

def Add_Word_To_Table(list_2d):
    table = copy.deepcopy(list_2d)
    table.insert(1, [i for i in " " + WORD])
    table[1][0] = ""
    
    return table

def Add_Lines(list_2d):
    table = copy.deepcopy(list_2d)
    for i in range(len(table)):
        for j in range(len(table) - i):
            if table[i][j] != []:
                for k in range(1, i):
                    if table[i-k][j+k] == []:
                        table[i-k][j+k] = ['∕']
                    else:
                        break
                for k in range(1, i):
                    if table[i-k][j] == []:
                        table[i-k][j] = ['|']
                    else:
                        break
    return table

In [2]:
class KEK(object):
    
    def __init__(self, dict_of_nonterminals_to_nonterminals, dict_of_nonterminals_to_terminals, word):
        self.inv_n_to_n = Invert_Dict(dict_of_nonterminals_to_nonterminals)
        self.inv_n_to_t = Invert_Dict(dict_of_nonterminals_to_terminals)
        self.building_list = [[[] for j in range(len(word) - i)] for i in range(len(word))] # треугольная сетка
        self.list_of_building_moves = [[[] for j in range(len(word)- i)] for i in range(len(word))] # еще одна для того чтобы показать из чего мы строили предыдущую сетку
        for i in range(len(word)):
            self.building_list[0][i] = self.inv_n_to_t[word[i]] #нетерминалы которые выводят терминалы
        self.word = word
        self.move_tables = []
        
    def Run_KEK(self):
        for y in range(0, len(self.word)-1): #страшно Тём.. вырубай..    "y","x" - координаты сетки
            for x in range(0, len(self.word)-1 - y):                   # "i" - первая цифра из списка МИНУС ОДИН (1 7, 2 6, 3 5, 4 4...)
                for i in range(0, y+1):                         # "j","k" - буквы которые взяты перебором из 2ух клеток сетки
                    left = self.building_list[i][x]
                    right = self.building_list[y-i][i+x+1]
                    for j in range(len(left)):               # "l" - мне показалось что циклов недостаточно (а вообще проходимся по списку правил взятого из словаря для букв j+k)
                        for k in range(len(right)):
                            if left[j] + right[k] in self.inv_n_to_n:
                                for l in self.inv_n_to_n[left[j] + right[k]]:
                                    self.building_list[y+1][x].append(l)
                                    self.list_of_building_moves[y+1][x].append([left[j], i, x, j, right[k], y-i, i+x+1, k]) #позиции в списке и их буквы для родительских правил
    
    def Run_Moves(self):
        def recursive_search(building_list, moves, y, x, pos_current, table):
            
            if building_list[y][x]:
                table[y][x] = building_list[y][x][pos_current]
            if moves[y][x]:
                recursive_search(building_list, moves, moves[y][x][pos_current][1], moves[y][x][pos_current][2], moves[y][x][pos_current][3], table)
                recursive_search(building_list, moves, moves[y][x][pos_current][5], moves[y][x][pos_current][6], moves[y][x][pos_current][7], table)
        
        for k in range(len(self.list_of_building_moves[len(self.word)-1][0])):
            new_move_table = [[[] for j in range(len(self.word) - i)] for i in range(len(self.word))]
            recursive_search(self.building_list, self.list_of_building_moves, len(self.word)-1, 0, k, new_move_table)
            self.move_tables.append(new_move_table)
    
    def Print_KEK(self):
        Print_2D_Table(Add_Word_To_Table(Add_Row_Column_Numeration(self.building_list, start_col=1, start_row=1)))
        
    def Print_Moves(self):
        for i in self.move_tables:
            Print_2D_Table(Add_Word_To_Table(Add_Row_Column_Numeration(Add_Lines(i), start_col=1, start_row=1)))

In [5]:
DICT_OF_NONTERMINALS_TO_NONTERMINALS = {
    "C" : ["EA"],
    "D" : ["EB"],
    "H" : ["FE"],
    "E" : ["CE", "DE", "HG"]
}

DICT_OF_NONTERMINALS_TO_TERMINALS = {
    "A" : ["+"],
    "B" : ["*"],
    "E" : ["a"],
    "F" : ["("],
    "G" : [")"]
}

#второй вариант
WORD = "(a+a)*a"

test = KEK(DICT_OF_NONTERMINALS_TO_NONTERMINALS, DICT_OF_NONTERMINALS_TO_TERMINALS, WORD)
test.Run_KEK()
test.Print_KEK()
test.Run_Moves()
test.Print_Moves()

╒═══╤═══╤═══╤═══╤═══╤═══╤═══╤═══╕
│   │ 1 │ 2 │ 3 │ 4 │ 5 │ 6 │ 7 │
├───┼───┼───┼───┼───┼───┼───┼───┤
│   │ ( │ a │ + │ a │ ) │ * │ a │
├───┼───┼───┼───┼───┼───┼───┼───┤
│ 1 │ F │ E │ A │ E │ G │ B │ E │
├───┼───┼───┼───┼───┼───┼───┼───┤
│ 2 │ H │ C │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┤
│ 3 │   │ E │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┤
│ 4 │ H │   │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┤
│ 5 │ E │   │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┤
│ 6 │ D │   │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┤
│ 7 │ E │   │   │   │   │   │   │
╘═══╧═══╧═══╧═══╧═══╧═══╧═══╧═══╛
╒═══╤═══╤═══╤═══╤═══╤═══╤═══╤═══╕
│   │ 1 │ 2 │ 3 │ 4 │ 5 │ 6 │ 7 │
├───┼───┼───┼───┼───┼───┼───┼───┤
│   │ ( │ a │ + │ a │ ) │ * │ a │
├───┼───┼───┼───┼───┼───┼───┼───┤
│ 1 │ F │ E │ A │ E │ G │ B │ E │
├───┼───┼───┼───┼───┼───┼───┼───┤
│ 2 │ | │ C │ ∕ │ ∕ │ ∕ │ ∕ │   │
├───┼───┼───┼───┼───┼───┼───┼───┤
│ 3 │ | │ E │ ∕ │ ∕ │ ∕ │   │   │
├───┼───┼───┼─

In [6]:
DICT_OF_NONTERMINALS_TO_NONTERMINALS = {
    "D" : ["AS"],
    "E" : ["CS"],
    "S" : ["AB", "DB", "AC", "DC", "CB", "EB"]
}

DICT_OF_NONTERMINALS_TO_TERMINALS = {
    "A" : ["a"],
    "B" : ["b"],
    "C" : ["c"]
}

WORD = "aaacbbcb"

test = KEK(DICT_OF_NONTERMINALS_TO_NONTERMINALS, DICT_OF_NONTERMINALS_TO_TERMINALS, WORD)
test.Run_KEK()
test.Print_KEK()
test.Run_Moves()
test.Print_Moves()

╒═══╤═══╤═══╤═══╤═══╤═══╤═══╤═══╤═══╕
│   │ 1 │ 2 │ 3 │ 4 │ 5 │ 6 │ 7 │ 8 │
├───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │ a │ a │ a │ c │ b │ b │ c │ b │
├───┼───┼───┼───┼───┼───┼───┼───┼───┤
│ 1 │ A │ A │ A │ C │ B │ B │ C │ B │
├───┼───┼───┼───┼───┼───┼───┼───┼───┤
│ 2 │   │   │ S │ S │   │   │ S │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┤
│ 3 │   │ D │ D │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┤
│ 4 │   │ S │ S │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┤
│ 5 │ D │ D │   │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┤
│ 6 │ S │ S │   │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┤
│ 7 │ D │   │   │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┤
│ 8 │ S │   │   │   │   │   │   │   │
╘═══╧═══╧═══╧═══╧═══╧═══╧═══╧═══╧═══╛
╒═══╤═══╤═══╤═══╤═══╤═══╤═══╤═══╤═══╕
│   │ 1 │ 2 │ 3 │ 4 │ 5 │ 6 │ 7 │ 8 │
├───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │ a │ a │ a │ c │ b │ b │ c │ b │
├───┼───┼───┼───┼───┼───┼───┼───┼───┤
│ 1 │ A │ A 

In [7]:
DICT_OF_NONTERMINALS_TO_NONTERMINALS = {
    "P" : ["AB", "BG"],
    "A" : ["ZA"],
    "B" : ["YB"],
    "G" : ["YA"]
}

DICT_OF_NONTERMINALS_TO_TERMINALS = {
    "Z" : ["a"],
    "Y" : ["b"],
    "B" : ["c"],
    "G" : ["c", "b"],
    "P" : ["c"],
    "A" : ["a"]
}

WORD = "bbbcaaa"

test = KEK(DICT_OF_NONTERMINALS_TO_NONTERMINALS, DICT_OF_NONTERMINALS_TO_TERMINALS, WORD)
test.Run_KEK()
test.Print_KEK()
test.Run_Moves()
test.Print_Moves()

╒═══╤═════╤═════╤═════╤═══════╤═════╤═════╤═════╕
│   │  1  │  2  │  3  │   4   │  5  │  6  │  7  │
├───┼─────┼─────┼─────┼───────┼─────┼─────┼─────┤
│   │  b  │  b  │  b  │   c   │  a  │  a  │  a  │
├───┼─────┼─────┼─────┼───────┼─────┼─────┼─────┤
│ 1 │ Y,G │ Y,G │ Y,G │ B,G,P │ Z,A │ Z,A │ Z,A │
├───┼─────┼─────┼─────┼───────┼─────┼─────┼─────┤
│ 2 │     │     │  B  │       │  A  │  A  │     │
├───┼─────┼─────┼─────┼───────┼─────┼─────┼─────┤
│ 3 │     │  B  │     │       │  A  │     │     │
├───┼─────┼─────┼─────┼───────┼─────┼─────┼─────┤
│ 4 │  B  │     │     │       │     │     │     │
├───┼─────┼─────┼─────┼───────┼─────┼─────┼─────┤
│ 5 │     │     │     │       │     │     │     │
├───┼─────┼─────┼─────┼───────┼─────┼─────┼─────┤
│ 6 │     │     │     │       │     │     │     │
├───┼─────┼─────┼─────┼───────┼─────┼─────┼─────┤
│ 7 │     │     │     │       │     │     │     │
╘═══╧═════╧═════╧═════╧═══════╧═════╧═════╧═════╛


In [9]:
DICT_OF_NONTERMINALS_TO_NONTERMINALS = {
    "X" : ["ZY"],
    "B" : ["XS"],
    "D" : ["ZZ"],
    "E" : ["DA"],
    "G" : ["YZ"],
    "H" : ["GA"],
    "S" : ["BZ", "EY"],
    "A" : ["HY"]
}

DICT_OF_NONTERMINALS_TO_TERMINALS = {
    "Z" : ["a"],
    "Y" : ["b"],
    "S" : ["b"],
    "A" : ["b"]
}

WORD = "abaababbba"

test = KEK(DICT_OF_NONTERMINALS_TO_NONTERMINALS, DICT_OF_NONTERMINALS_TO_TERMINALS, WORD)
test.Run_KEK()
test.Print_KEK()
test.Run_Moves()
test.Print_Moves()

╒════╤═══╤═══════╤═══╤═══╤═══════╤═══╤═══════╤═══════╤═══════╤════╕
│    │ 1 │   2   │ 3 │ 4 │   5   │ 6 │   7   │   8   │   9   │ 10 │
├────┼───┼───────┼───┼───┼───────┼───┼───────┼───────┼───────┼────┤
│    │ a │   b   │ a │ a │   b   │ a │   b   │   b   │   b   │ a  │
├────┼───┼───────┼───┼───┼───────┼───┼───────┼───────┼───────┼────┤
│ 1  │ Z │ Y,S,A │ Z │ Z │ Y,S,A │ Z │ Y,S,A │ Y,S,A │ Y,S,A │ Z  │
├────┼───┼───────┼───┼───┼───────┼───┼───────┼───────┼───────┼────┤
│ 2  │ X │   G   │ D │ X │   G   │ X │       │       │   G   │    │
├────┼───┼───────┼───┼───┼───────┼───┼───────┼───────┼───────┼────┤
│ 3  │   │       │ E │   │   H   │ B │       │       │       │    │
├────┼───┼───────┼───┼───┼───────┼───┼───────┼───────┼───────┼────┤
│ 4  │   │       │   │   │   A   │   │       │       │       │    │
├────┼───┼───────┼───┼───┼───────┼───┼───────┼───────┼───────┼────┤
│ 5  │   │       │   │   │       │   │       │       │       │    │
├────┼───┼───────┼───┼───┼───────┼───┼───────┼──

In [10]:
DICT_OF_NONTERMINALS_TO_NONTERMINALS = {
    "S" : ["BY", "EY", "BD", "CE", "FD", "AB", "GD", "AF", "HD", "BS", "ID", "AB"],
    "A" : ["ZC", "ZA", "ZJ"],
    "B" : ["BY", "EY"],
    "C" : ["PC"],
    "P" : ["XC", "XD", "XK"],
    "E" : ["BC"],
    "F" : ["CE"],
    "G" : ["AB"],
    "H" : ["AF"],
    "I" : ["BS"],
    "J" : ["CA"],
    "K" : ["CD"]
}

DICT_OF_NONTERMINALS_TO_TERMINALS = {
    "Z" : ["к"],
    "Y" : ["н"],
    "X" : ["п"],
    "P" : [","],
    "S" : ["н"],
    "A" : ["к"],
    "B" : ["н"],
    "C" : [","],
    "P" : ["п"]
}

WORD = "к,нп"

test = KEK(DICT_OF_NONTERMINALS_TO_NONTERMINALS, DICT_OF_NONTERMINALS_TO_TERMINALS, WORD)
test.Run_KEK()
test.Print_KEK()
test.Run_Moves()
test.Print_Moves()

╒═══╤═══════╤═══╤═══════╤═════╕
│   │   1   │ 2 │   3   │  4  │
├───┼───────┼───┼───────┼─────┤
│   │   к   │ , │   н   │  п  │
├───┼───────┼───┼───────┼─────┤
│ 1 │  Z,A  │ C │ Y,S,B │ X,P │
├───┼───────┼───┼───────┼─────┤
│ 2 │   A   │   │       │     │
├───┼───────┼───┼───────┼─────┤
│ 3 │ S,S,G │   │       │     │
├───┼───────┼───┼───────┼─────┤
│ 4 │       │   │       │     │
╘═══╧═══════╧═══╧═══════╧═════╛
